# Wildfire Impact Analysis on Larimer County, Colorado 

In [1]:
import sys, json
from Reader import Reader
import time
import matplotlib.pyplot as plt
import numpy as np
import math 
from shapely.geometry import Polygon, Point
from math import radians, sin, cos, sqrt, atan2
import re
from datetime import datetime
from collections import Counter
from shapely.ops import nearest_points

## Loading in USGS Fire Data
Data was taken from the [USGS Website](https://www.sciencebase.gov/catalog/item/61aa537dd34eb622f699df81). This data is the combined fire history for all US states since the late 1800s.

In [11]:
WF = 'USGS_Wildland_Fire_Combined_Dataset.json'
wf_reader = Reader(WF)
header = wf_reader.header()
print(header)

{'displayFieldName': '', 'fieldAliases': {'OBJECTID': 'OBJECTID', 'USGS_Assigned_ID': 'USGS Assigned ID', 'Assigned_Fire_Type': 'Assigned Fire Type', 'Fire_Year': 'Fire Year', 'Fire_Polygon_Tier': 'Fire Polygon Tier', 'Fire_Attribute_Tiers': 'Fire Attribute Tiers', 'GIS_Acres': 'GIS_Acres', 'GIS_Hectares': 'GIS_Hectares', 'Source_Datasets': 'Source Datasets', 'Listed_Fire_Types': 'Listed Fire Types', 'Listed_Fire_Names': 'Listed Fire Names', 'Listed_Fire_Codes': 'Listed Fire Codes', 'Listed_Fire_IDs': 'Listed Fire IDs', 'Listed_Fire_IRWIN_IDs': 'Listed Fire IRWIN IDs', 'Listed_Fire_Dates': 'Listed Fire Dates', 'Listed_Fire_Causes': 'Listed Fire Causes', 'Listed_Fire_Cause_Class': 'Listed Fire Cause Class', 'Listed_Rx_Reported_Acres': 'Listed Rx Reported Acres', 'Listed_Map_Digitize_Methods': 'Listed Map Digitize Methods', 'Listed_Notes': 'Listed Notes', 'Processing_Notes': 'Processing Notes', 'Wildfire_Notice': 'Wildfire Notice', 'Prescribed_Burn_Notice': 'Prescribed Burn Notice', 'Wil

In [26]:
found_count = 0
feature_count = 0
assigned_city = (40.3955, -105.0746)  # Latitude, Longitude
assigned_city_point = Point(assigned_city[1], assigned_city[0])
feature_list = []

def convert_ring_to_epsg4326(ring_data=None):
    converted_ring = list()
    #
    # We use a pyproj transformer that converts from ESRI:102008 to EPSG:4326 to transform the list of coordinates
    to_epsg4326 = Transformer.from_crs("ESRI:102008","EPSG:4326")
    # We'll run through the list transforming each ESRI:102008 x,y coordinate into a decimal degree lat,lon
    for coord in ring_data:
        lat,lon = to_epsg4326.transform(coord[0],coord[1])
        new_coord = lat,lon
        converted_ring.append(new_coord)
    return converted_ring

#    
#    The function takes two parameters
#        A place - which is coordinate point (list or tuple with two items, (lat,lon) in decimal degrees EPSG:4326
#        Ring_data - a list of decimal degree coordinates for the fire boundary
#
#    The function returns a list containing the shortest distance to the perimeter and the point where that is
#
def shortest_distance_from_place_to_fire_perimeter(place=None,ring_data=None):
    # convert the ring data to the right coordinate system
    ring = convert_ring_to_epsg4326(ring_data)    
    # create a epsg4326 compliant object - which is what the WGS84 ellipsoid is
    geodcalc = Geod(ellps='WGS84')
    closest_point = list()
    # run through each point in the converted ring data
    for point in ring:
        # calculate the distance
        d = geodcalc.inv(place[1],place[0],point[1],point[0])
        # convert the distance to miles
        distance_in_miles = d[2]*0.00062137
        # if it's closer to the city than the point we have, save it
        if not closest_point:
            closest_point.append(distance_in_miles)
            closest_point.append(point)
        elif closest_point and closest_point[0]>distance_in_miles:
            closest_point = list()
            closest_point.append(distance_in_miles)
            closest_point.append(point)
    return closest_point



## Grabbing needed data from USGS
For our analysis, we only want fires that fit this criteria: must be a wildfire, must have occured between 1963 and 2023, and must be within 1250 miles of our city (Larimer County). Distance from said fire is calculated by the centriod of the fire, not the most outer ring.  

In [31]:
import time
wf_reader = Reader(WF)
feature = wf_reader.next()
feature_count = 0
found_count = 0
while feature:
    feature_count += 1
    
    fire_year = int(feature['attributes']['Fire_Year'])
    fire_type = feature['attributes']['Listed_Fire_Types'].lower()
    wf = 'wildfire' in fire_type
    if 1963 <= fire_year <= 2023 and wf:
        # Filter by Distance
        geometry = feature.get('geometry', {})
        if 'rings' in geometry:
            ring_data = geometry['rings'][0]
            distance = shortest_distance_from_place_to_fire_perimeter([40.3955, -105.0746],ring_data)
            if distance[0] <= 1250:
                found_count += 1
                if found_count % 100 == 0:
                    print(found_count)
                feature['fire_distance'] = distance
                feature_list.append(feature)  # Storing the filtered feature

    feature = wf_reader.next()

print(f"Total Features: {feature_count}")
print(f"Features meeting criteria: {found_count}")

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [32]:
# saving data to a json file
with open('data/filtered_features_final.json', 'w') as f:
    json.dump(feature_list, f)

## Making Smoke Severity Estimate 
This smoke severity estimate should represent how severe the wildfire smoke will be in our County. Our estimate is calcuated by dividing the total area of the fire by the square root of the distance from our County. 

In [36]:
from collections import defaultdict

# dictionary to hold the annual smoke estimates
annual_smoke_estimate = defaultdict(float)

for feature in feature_list:
    fire_year = feature['attributes']['Fire_Year']
    fire_miles = feature['attributes']['GIS_Acres'] * 0.0015625 # converting acres to miles
    
    geometry = feature.get('geometry', {})
    if 'rings' in geometry:
        ring_data = geometry['rings'][0]
        distance = shortest_distance_from_place_to_fire_perimeter([40.3955, -105.0746],ring_data)
        
        # formula to estimate smoke for this particular fire
        if distance[0] > 0:  # avoid division by zero
            smoke_estimate = fire_miles / (distance[0] ** 2) 
            
            # add the smoke estimates annually
            annual_smoke_estimate[fire_year] += smoke_estimate

annual_smoke_estimate

defaultdict(float,
            {1963: 0.0023773497618277315,
             1964: 0.0020043115695999613,
             1965: 0.0009777136083809364,
             1966: 0.003317580383719445,
             1967: 0.0014363065206335502,
             1968: 0.000896884522702971,
             1969: 0.0010043379605474665,
             1970: 0.0024946030845441306,
             1971: 0.004332160373283769,
             1972: 0.004078833632420163,
             1973: 0.0024630663892897596,
             1974: 0.0021163149167690094,
             1975: 0.0018615007423819822,
             1976: 0.002547898052789835,
             1977: 0.002184987298155798,
             1978: 0.0019311321276157818,
             1979: 0.003141337722125578,
             1980: 0.007149889136786897,
             1981: 0.006768566093351206,
             1982: 0.001972923206812418,
             1983: 0.004611516607021052,
             1984: 0.0061189137402432505,
             1985: 0.014650093598823499,
             1986: 0.009991

## Using AQI data from EPA
In the file `EPA_AQI_DATA_LOAD.ipynb` we show how we grabbed this data. This section focuses on formating it, and reducing it into usable data. This section below combines each type of AQI estimate into aggregate data based upon the year.

In [37]:
import numpy as np

# Load AQI and smoke data
with open('data/gaseous_aqi_data.json', 'r') as f:
    gaseous_aqi_data = json.load(f)
with open('data/particulate_aqi_data.json', 'r') as f:
    particulate_aqi_data = json.load(f)
aggregated_data = {}

# loops through every year, get the pollutant type and AQI value
for year, year_data in gaseous_aqi_data.items():
    for site_id, site_data in year_data.items():
        for pollutant_id, pollutant_data in site_data['pollutant_type'].items():
            for date, measurements in pollutant_data['data'].items():
                year = date[:4]
                if year not in aggregated_data:
                    aggregated_data[year] = {}
                if pollutant_id not in aggregated_data[year]:
                    aggregated_data[year][pollutant_id] = []
                for measurement in measurements:
                    aqi_value = measurement.get('aqi', None)
                    if aqi_value is not None:
                        aggregated_data[year][pollutant_id].append(aqi_value)

for year, year_data in particulate_aqi_data.items():
    for site_id, site_data in year_data.items():
        for pollutant_id, pollutant_data in site_data['pollutant_type'].items():
            for date, measurements in pollutant_data['data'].items():
                year = date[:4]
                if year not in aggregated_data:
                    aggregated_data[year] = {}
                if pollutant_id not in aggregated_data[year]:
                    aggregated_data[year][pollutant_id] = []
                for measurement in measurements:
                    aqi_value = measurement.get('aqi', None)
                    if aqi_value is not None:
                        aggregated_data[year][pollutant_id].append(aqi_value)


# Calculate average AQI per year for each pollutant
average_aqi_per_year = defaultdict(dict)

for year, pollutants in aggregated_data.items():
    for pollutant_id, aqi_values in pollutants.items():
        average_aqi = np.mean(aqi_values)
        average_aqi_per_year[year][pollutant_id] = round(average_aqi, 2)

average_aqi_per_year

/home/peter/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/peter/.local/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


defaultdict(dict,
            {'1975': {'42101': 81.2, '44201': 116.77},
             '1976': {'42101': 24.27},
             '1978': {'44201': 60.82},
             '1979': {'44201': 58.12},
             '1980': {'42101': 30.2, '44201': 40.35},
             '1981': {'42101': 25.96, '44201': 39.94},
             '1982': {'42101': 21.88, '44201': 37.42},
             '1983': {'42101': 22.28, '44201': 44.64},
             '1984': {'42101': 21.38, '44201': 42.02},
             '1985': {'42101': 24.79, '44201': 39.96},
             '1986': {'42101': 22.12, '44201': 39.08, '81102': 28.23},
             '1987': {'42101': 20.66, '44201': 45.6, '81102': 26.54},
             '1988': {'42101': 22.32,
              '44201': 51.44,
              '81102': 23.79,
              '88502': 17.33},
             '1989': {'42101': 21.32,
              '44201': 47.99,
              '81102': 28.25,
              '88502': 19.37},
             '1990': {'42101': 17.91,
              '44201': 41.99,
              

In [39]:
# re-naming so the data is more human-readable
pollutant_code_mapping = {
    '42101': 'CO',
    '42401': 'SO2',
    '42602': 'NO2',
    '44201': 'O2',
    '81102': 'PM10',
    '88101': 'PM2.5',
    '88502': 'Acceptable PM2.5'
}

# rename keys in the dictionary based on the mapping
def rename_keys(dictionary, mapping):
    return {mapping.get(key, key): value for key, value in dictionary.items()}

renamed_aggregated_data = {year: rename_keys(year_data, pollutant_code_mapping) for year, year_data in average_aqi_per_year.items()}

renamed_aggregated_data


{'1975': {'CO': 81.2, 'O2': 116.77},
 '1976': {'CO': 24.27},
 '1978': {'O2': 60.82},
 '1979': {'O2': 58.12},
 '1980': {'CO': 30.2, 'O2': 40.35},
 '1981': {'CO': 25.96, 'O2': 39.94},
 '1982': {'CO': 21.88, 'O2': 37.42},
 '1983': {'CO': 22.28, 'O2': 44.64},
 '1984': {'CO': 21.38, 'O2': 42.02},
 '1985': {'CO': 24.79, 'O2': 39.96},
 '1986': {'CO': 22.12, 'O2': 39.08, 'PM10': 28.23},
 '1987': {'CO': 20.66, 'O2': 45.6, 'PM10': 26.54},
 '1988': {'CO': 22.32, 'O2': 51.44, 'PM10': 23.79, 'Acceptable PM2.5': 17.33},
 '1989': {'CO': 21.32, 'O2': 47.99, 'PM10': 28.25, 'Acceptable PM2.5': 19.37},
 '1990': {'CO': 17.91, 'O2': 41.99, 'PM10': 21.67, 'Acceptable PM2.5': 20.15},
 '1991': {'CO': 12.39,
  'O2': 49.27,
  'SO2': nan,
  'PM10': 20.48,
  'Acceptable PM2.5': 16.81},
 '1992': {'CO': 13.78, 'O2': 46.29, 'PM10': 20.92, 'Acceptable PM2.5': 19.06},
 '1993': {'CO': 11.59,
  'O2': 44.91,
  'SO2': nan,
  'PM10': 17.32,
  'Acceptable PM2.5': 16.91},
 '1994': {'CO': 12.99,
  'O2': 48.76,
  'SO2': nan,
 

## Choosing our AQI estimates 
For this analysis, we are only taking into account 3 types of measurements: pm10, pm2.5, and CO. PM2.5 and PM10(sort for particulate matter < 2.5 micrometers and < 10 micrometers) best represent the kind of matter that a wildfire produces. Wildfires also greatly produce CO (short for Carbon Monoxide). 

In [40]:
# Extract PM2.5 and pm10 data for each year
pm25_data = {}
ozone_data = {}
pm10_data = {}
co_data = {}

pm25_id = 'PM2.5'
ozone_id = 'O2' 
pm10_id = 'PM10'
co_id = 'CO'

# Extract PM2.5 data
for year, data in renamed_aggregated_data.items():
    if pm25_id in data:
        pm25_data[year] = data[pm25_id]
        
# Extract PM10 data       
for year, data in renamed_aggregated_data.items():
    if pm10_id in data:
        pm10_data[year] = data[pm10_id]
        
# Extract CO data
for year, data in renamed_aggregated_data.items():
    if co_id in data:
        co_data[year] = data[co_id]
    
comparison_data = {}
for year, estimate in annual_smoke_estimate.items():
    comparison_data[year] = {
        'estimated': estimate,
        'pm25': pm25_data.get(str(year), None),
 #       'ozone': ozone_data.get(str(year), None),
        'pm10': pm10_data.get(str(year), None),
        'CO': co_data.get(str(year), None)
    }
print(comparison_data)

{1963: {'estimated': 0.0023773497618277315, 'pm25': None, 'pm10': None, 'CO': None}, 1964: {'estimated': 0.0020043115695999613, 'pm25': None, 'pm10': None, 'CO': None}, 1965: {'estimated': 0.0009777136083809364, 'pm25': None, 'pm10': None, 'CO': None}, 1966: {'estimated': 0.003317580383719445, 'pm25': None, 'pm10': None, 'CO': None}, 1967: {'estimated': 0.0014363065206335502, 'pm25': None, 'pm10': None, 'CO': None}, 1968: {'estimated': 0.000896884522702971, 'pm25': None, 'pm10': None, 'CO': None}, 1969: {'estimated': 0.0010043379605474665, 'pm25': None, 'pm10': None, 'CO': None}, 1970: {'estimated': 0.0024946030845441306, 'pm25': None, 'pm10': None, 'CO': None}, 1971: {'estimated': 0.004332160373283769, 'pm25': None, 'pm10': None, 'CO': None}, 1972: {'estimated': 0.004078833632420163, 'pm25': None, 'pm10': None, 'CO': None}, 1973: {'estimated': 0.0024630663892897596, 'pm25': None, 'pm10': None, 'CO': None}, 1974: {'estimated': 0.0021163149167690094, 'pm25': None, 'pm10': None, 'CO': No